# 2e -- Running Average Features



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
import winsound
import random
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')


### Analyze the Boxscore Dataframe that we will be using to solve 

In [2]:
df = pd.read_csv('data/team/All_Things_4.csv')
df.head(50)

,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,trad_fg%,trad_3pm,...,net_score_top_4,net_score_top_5,net_score_top_6,net_score_top_7,net_score_top_8,top_4_predicted_winner,top_5_predicted_winner,top_6_predicted_winner,top_7_predicted_winner,top_8_predicted_winner
0,CHI,CHI @ NYK,2022-12-23,W,48.0,118.0,45.0,93.0,0.484,14.0,...,-0.080000,-0.020000,0.050000,0.050000,0.122071,0,0,1,1,1
1,NYK,NYK vs. CHI,2022-12-23,L,48.0,117.0,43.0,86.0,0.500,16.0,...,0.080000,0.020000,-0.050000,-0.050000,-0.122071,1,1,0,0,0
2,TOR,TOR @ CLE,2022-12-23,W,48.0,118.0,42.0,84.0,0.500,19.0,...,-0.295857,-0.413786,-0.491714,-0.571714,-0.721714,0,0,0,0,0
3,PHI,PHI vs. LAC,2022-12-23,W,48.0,119.0,47.0,88.0,0.534,12.0,...,-0.300000,-0.360000,-0.470000,-0.530000,-0.530000,0,0,0,0,0
4,MIA,MIA vs. IND,2022-12-23,L,48.0,108.0,41.0,95.0,0.432,13.0,...,0.250000,0.427929,0.575857,0.643786,0.671714,1,1,1,1,1
5,IND,IND @ MIA,2022-12-23,W,48.0,111.0,40.0,83.0,0.482,21.0,...,-0.250000,-0.427929,-0.575857,-0.643786,-0.671714,0,0,0,0,0
6,MIL,MIL @ BKN,2022-12-23,L,48.0,100.0,37.0,84.0,0.440,9.0,...,1.087929,1.265857,1.333786,1.391714,1.423786,1,1,1,1,1
7,BKN,BKN vs. MIL,2022-12-23,W,48.0,118.0,46.0,93.0,0.495,14.0,...,-1.087929,-1.265857,-1.333786,-1.391714,-1.423786,0,0,0,0,0
8,POR,POR @ DEN,2022-12-23,L,48.0,107.0,39.0,87.0,0.448,9.0,...,0.280000,0.320000,0.337929,0.337929,0.337929,1,1,1,1,1
9,LAL,LAL vs. CHA,2022-12-23,L,48.0,130.0,48.0,92.0,0.522,19.0,...,0.857929,1.025857,1.113786,1.125857,1.157929,1,1,1,1,1


In [3]:
# Count index errors in top_4_predicted_winner
df['top_5_predicted_winner'].value_counts()

0    9890
1    6208
Name: top_5_predicted_winner, dtype: int64

In [4]:
# TODO: Figure out where these came from! They could be meaningful. 

# df = df.drop(df[df['net_score_top_4'] == 'IndexError'].index)
# df = df.drop(df[df['net_score_top_5'] == 'IndexError'].index)
# df = df.drop(df[df['net_score_top_6'] == 'IndexError'].index)
# df = df.drop(df[df['net_score_top_7'] == 'IndexError'].index)

# df['net_score_top_4'] = df['net_score_top_4'].astype(np.float64)
# df['net_score_top_5'] = df['net_score_top_5'].astype(np.float64)
# df['net_score_top_6'] = df['net_score_top_6'].astype(np.float64)
# df['net_score_top_7'] = df['net_score_top_7'].astype(np.float64)

In [5]:
# df['net_score_top_4'] = df['net_score_top_4'].astype(np.float64)
# df['net_score_top_5'] = df['net_score_top_5'].astype(np.float64)
# df['net_score_top_6'] = df['net_score_top_6'].astype(np.float64)
# df['net_score_top_7'] = df['net_score_top_7'].astype(np.float64)

In [6]:
colz= list(df.columns)

In [7]:
df.head()

,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,trad_fg%,trad_3pm,...,net_score_top_4,net_score_top_5,net_score_top_6,net_score_top_7,net_score_top_8,top_4_predicted_winner,top_5_predicted_winner,top_6_predicted_winner,top_7_predicted_winner,top_8_predicted_winner
0,CHI,CHI @ NYK,2022-12-23,W,48.0,118.0,45.0,93.0,0.484,14.0,...,-0.080000,-0.020000,0.050000,0.050000,0.122071,0,0,1,1,1
1,NYK,NYK vs. CHI,2022-12-23,L,48.0,117.0,43.0,86.0,0.500,16.0,...,0.080000,0.020000,-0.050000,-0.050000,-0.122071,1,1,0,0,0
2,TOR,TOR @ CLE,2022-12-23,W,48.0,118.0,42.0,84.0,0.500,19.0,...,-0.295857,-0.413786,-0.491714,-0.571714,-0.721714,0,0,0,0,0
3,PHI,PHI vs. LAC,2022-12-23,W,48.0,119.0,47.0,88.0,0.534,12.0,...,-0.300000,-0.360000,-0.470000,-0.530000,-0.530000,0,0,0,0,0
4,MIA,MIA vs. IND,2022-12-23,L,48.0,108.0,41.0,95.0,0.432,13.0,...,0.250000,0.427929,0.575857,0.643786,0.671714,1,1,1,1,1


In [8]:
# fix gamedates
df['date'] = pd.to_datetime(df['trad_gamedate'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.sort_values('date', ascending = False, inplace=True)
df

,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,trad_fg%,trad_3pm,...,net_score_top_5,net_score_top_6,net_score_top_7,net_score_top_8,top_4_predicted_winner,top_5_predicted_winner,top_6_predicted_winner,top_7_predicted_winner,top_8_predicted_winner,date
0,CHI,CHI @ NYK,2022-12-23,W,48.0,118.0,45.0,93.0,0.484,14.0,...,-0.020000,0.050000,0.050000,0.122071,0,0,1,1,1,2022-12-23
15,DET,DET @ ATL,2022-12-23,L,48.0,105.0,37.0,90.0,0.411,9.0,...,-0.850000,-0.930000,-1.027929,-1.115857,0,0,0,0,0,2022-12-23
27,LAC,LAC @ PHI,2022-12-23,L,48.0,114.0,41.0,81.0,0.506,14.0,...,0.360000,0.470000,0.530000,0.530000,1,1,1,1,1,2022-12-23
26,ATL,ATL vs. DET,2022-12-23,W,48.0,130.0,49.0,92.0,0.533,12.0,...,0.850000,0.930000,1.027929,1.115857,1,1,1,1,1,2022-12-23
25,MEM,MEM @ PHX,2022-12-23,W,48.0,125.0,45.0,91.0,0.495,7.0,...,0.022071,0.022071,0.022071,0.022071,1,1,1,1,1,2022-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16093,NYK,NYK @ CLE,2016-10-25,L,48.0,88.0,32.0,87.0,0.368,9.0,...,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,2016-10-25
16094,GSW,GSW vs. SAS,2016-10-25,L,48.0,100.0,40.0,85.0,0.471,7.0,...,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,2016-10-25
16095,UTA,UTA @ POR,2016-10-25,L,48.0,104.0,40.0,82.0,0.488,8.0,...,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,2016-10-25
16096,CLE,CLE vs. NYK,2016-10-25,W,48.0,117.0,45.0,94.0,0.479,13.0,...,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,2016-10-25


In [9]:
df = df[df['adv_season_type'] == 'Regular']

In [10]:
#team_games_2016
games = df['Matchup_GameDate'].unique()
games

array(['CHI @ NYK_12_23_2022', 'DET @ ATL_12_23_2022',
       'LAC @ PHI_12_23_2022', ..., 'UTA @ POR_10_25_2016',
       'CLE vs. NYK_10_25_2016', 'POR vs. UTA_10_25_2016'], dtype=object)

In [11]:
def get_game_avgs(my_gameid, team, season, date, metric, game_num, agg_metric):
    data = df[df['trad_team'] == team]
    # filter by date
    data = data[data['date'] < date]
    # only last 80 games
    data = data.sort_values('date', ascending = False)
    data = data.head(game_num)
    data = data.dropna(subset = [metric])
    if agg_metric == 'std':
        metric = data[metric].std()
    else:
        metric = data[metric].mean()
    return metric

In [12]:
df['tm1_80_game_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'avg'), axis=1)
df['tm1_80_game_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'std'), axis=1)
df['tm1_80_game_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 80, 'avg'), axis=1)
df['tm1_80_game_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 80, 'std'), axis=1)
df['tm1_80_game_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 80, 'avg'), axis=1)
df['tm1_80_game_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 80, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\896359505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_80_game_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\896359505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_80_game_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], r

In [13]:
df['tm2_80_game_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'avg'), axis=1)
df['tm2_80_game_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'std'), axis=1)
df['tm2_80_game_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 80, 'avg'), axis=1)
df['tm2_80_game_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 80, 'std'), axis=1)
df['tm2_80_game_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 80, 'avg'), axis=1)
df['tm2_80_game_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 80, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\2062364640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_80_game_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\2062364640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_80_game_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['

### 40-game OffRtg, DefRtg, Pace

In [14]:
df['tm1_40_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'avg'), axis=1)
df['tm1_40_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'std'), axis=1)
df['tm1_40_gm_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 40, 'avg'), axis=1)
df['tm1_40_gm_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 40, 'std'), axis=1)
df['tm1_40_gm_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 40, 'avg'), axis=1)
df['tm1_40_gm_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 40, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\910363691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_40_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\910363691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_40_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['

In [15]:
df['tm2_40_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'avg'), axis=1)
df['tm2_40_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'std'), axis=1)
df['tm2_40_gm_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 40, 'avg'), axis=1)
df['tm2_40_gm_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 40, 'std'), axis=1)
df['tm2_40_gm_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 40, 'avg'), axis=1)
df['tm2_40_gm_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 40, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\704168315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_40_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\704168315.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_40_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_s

### 20-game OffRtg, DefRtg, Pace

In [16]:
df['tm1_20_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'avg'), axis=1)
df['tm1_20_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'std'), axis=1)
df['tm1_20_gm_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 20, 'avg'), axis=1)
df['tm1_20_gm_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 20, 'std'), axis=1)
df['tm1_20_gm_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 20, 'avg'), axis=1)
df['tm1_20_gm_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 20, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\4185467542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_20_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\4185467542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_20_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row

In [17]:
df['tm2_20_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'avg'), axis=1)
df['tm2_20_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'std'), axis=1)
df['tm2_20_gm_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 20, 'avg'), axis=1)
df['tm2_20_gm_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 20, 'std'), axis=1)
df['tm2_20_gm_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 20, 'avg'), axis=1)
df['tm2_20_gm_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 20, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\4060724046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_20_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\4060724046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_20_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad

In [18]:
df['tm1_10_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'avg'), axis=1)
df['tm1_10_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'std'), axis=1)
df['tm1_10_gm_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 10, 'avg'), axis=1)
df['tm1_10_gm_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_defrtg', 10, 'std'), axis=1)
df['tm1_10_gm_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 10, 'avg'), axis=1)
df['tm1_10_gm_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_pace', 10, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\2317676870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_10_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\2317676870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm1_10_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['trad_team'], row

In [19]:
df['tm2_10_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'avg'), axis=1)
df['tm2_10_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'std'), axis=1)
df['tm2_10_gm_avg_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 10, 'avg'), axis=1)
df['tm2_10_gm_std_defrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_defrtg', 10, 'std'), axis=1)
df['tm2_10_gm_avg_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 10, 'avg'), axis=1)
df['tm2_10_gm_std_pace'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_pace', 10, 'std'), axis=1)

C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\1996156412.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_10_gm_avg_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'avg'), axis=1)
C:\Users\Travis\AppData\Local\Temp\ipykernel_46672\1996156412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tm2_10_gm_std_offrtg'] = df.apply(lambda row: get_game_avgs(row['Matchup_GameDate'], row['team_2'], row['trad

In [20]:
df.to_csv('data/team/All_Things_5.csv')

In [21]:
df = pd.read_csv('data/team/All_Things_5.csv')

In [22]:
df.head()

,Unnamed: 0,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,trad_fg%,...,tm1_10_gm_avg_defrtg,tm1_10_gm_std_defrtg,tm1_10_gm_avg_pace,tm1_10_gm_std_pace,tm2_10_gm_avg_offrtg,tm2_10_gm_std_offrtg,tm2_10_gm_avg_defrtg,tm2_10_gm_std_defrtg,tm2_10_gm_avg_pace,tm2_10_gm_std_pace
0,0,CHI,CHI @ NYK,2022-12-23,W,48.0,118.0,45.0,93.0,0.484,...,114.43,13.100131,101.108,4.441616,114.78,12.487042,103.30,13.239000,97.369,4.045361
1,15,DET,DET @ ATL,2022-12-23,L,48.0,105.0,37.0,90.0,0.411,...,116.53,10.124124,100.143,3.079329,112.45,11.090361,116.64,8.139642,100.889,4.178901
2,27,LAC,LAC @ PHI,2022-12-23,L,48.0,114.0,41.0,81.0,0.506,...,107.58,12.277694,98.924,4.846073,111.06,9.975659,107.29,10.252203,98.748,5.058300
3,26,ATL,ATL vs. DET,2022-12-23,W,48.0,130.0,49.0,92.0,0.533,...,116.64,8.139642,100.889,4.178901,111.22,12.160300,116.53,10.124124,100.143,3.079329
4,25,MEM,MEM @ PHX,2022-12-23,W,48.0,125.0,45.0,91.0,0.495,...,102.73,5.123595,102.250,3.146868,116.58,13.802560,115.39,12.255289,97.955,3.574326


### Add Averages for 80, 40, and 20 [AND 10]

In [23]:
def get_NBA_avgs(my_gameid, season, date, metric, game_num, agg_metric):
    # filter by date
    data = df[df['date'] < date]
    # only last 80 games
    data = data.sort_values('date', ascending = False)
    gamess = game_num * 30 # for each team
    data = data.head(gamess)
    data = data.dropna(subset = [metric])
    if agg_metric == 'std':
        metric = data[metric].std()
    else:
        metric = data[metric].mean()
    return metric

#### Testing one

In [24]:
df['nba_80_gm_avg_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'avg'), axis=1)


##### NOTE: These take 45 minutes EACH to run (i.e, 3 hours total)

In [25]:
df['nba_80_gm_avg_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'avg'), axis=1)
df['nba_80_gm_std_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 80, 'std'), axis=1)
df['nba_80_gm_avg_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_defrtg', 80, 'avg'), axis=1)
df['nba_80_gm_std_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_defrtg', 80, 'std'), axis=1)
df['nba_80_gm_avg_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 80, 'avg'), axis=1)
df['nba_80_gm_std_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 80, 'std'), axis=1)

In [26]:
df['nba_40_gm_avg_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'avg'), axis=1)
df['nba_40_gm_std_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 40, 'std'), axis=1)
df['nba_40_gm_avg_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_defrtg', 40, 'avg'), axis=1)
df['nba_40_gm_std_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_defrtg', 40, 'std'), axis=1)
df['nba_40_gm_avg_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 40, 'avg'), axis=1)
df['nba_40_gm_std_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 40, 'std'), axis=1)

In [27]:
df['nba_20_gm_avg_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'avg'), axis=1)
df['nba_20_gm_std_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 20, 'std'), axis=1)
df['nba_20_gm_avg_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_defrtg', 20, 'avg'), axis=1)
df['nba_20_gm_std_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_defrtg', 20, 'std'), axis=1)
df['nba_20_gm_avg_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 20, 'avg'), axis=1)
df['nba_20_gm_std_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 20, 'std'), axis=1)

In [28]:
df['nba_10_gm_avg_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'avg'), axis=1)
df['nba_10_gm_std_offrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_offrtg', 10, 'std'), axis=1)
df['nba_10_gm_avg_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_defrtg', 10, 'avg'), axis=1)
df['nba_10_gm_std_defrtg'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'], row['trad_season'], row['date'], 'adv_defrtg', 10, 'std'), axis=1)
df['nba_10_gm_avg_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 10, 'avg'), axis=1)
df['nba_10_gm_std_pace'] = df.apply(lambda row: get_NBA_avgs(row['Matchup_GameDate'],  row['trad_season'], row['date'], 'adv_pace', 10, 'std'), axis=1)

In [29]:
df.to_csv('data/team/All_Things_6.csv')

### Team - NBA Averages

In [30]:
df = pd.read_csv('data/team/All_Things_6.csv')

In [31]:
df['80gm_tm1_minus_nba_avg_offrtg'] = df['tm1_80_game_avg_offrtg'] - df['nba_80_gm_avg_offrtg']
df['80gm_tm1_minus_nba_std_offrtg'] = df['tm1_80_game_std_offrtg'] - df['nba_80_gm_std_offrtg']
df['80gm_tm1_minus_nba_avg_defrtg'] = df['tm1_80_game_avg_defrtg'] - df['nba_80_gm_avg_defrtg']
df['80gm_tm1_minus_nba_std_defrtg'] = df['tm1_80_game_std_defrtg'] - df['nba_80_gm_std_defrtg']
df['80gm_tm1_minus_nba_avg_pace'] = df['tm1_80_game_avg_pace'] - df['nba_80_gm_avg_pace']
df['80gm_tm1_minus_nba_std_pace'] = df['tm1_80_game_std_pace'] - df['nba_80_gm_std_pace']

In [32]:
# also for team 2
df['80gm_tm2_minus_nba_avg_offrtg'] = df['tm2_80_game_avg_offrtg'] - df['nba_80_gm_avg_offrtg']
df['80gm_tm2_minus_nba_std_offrtg'] = df['tm2_80_game_std_offrtg'] - df['nba_80_gm_std_offrtg']
df['80gm_tm2_minus_nba_avg_defrtg'] = df['tm2_80_game_avg_defrtg'] - df['nba_80_gm_avg_defrtg']
df['80gm_tm2_minus_nba_std_defrtg'] = df['tm2_80_game_std_defrtg'] - df['nba_80_gm_std_defrtg']
df['80gm_tm2_minus_nba_avg_pace'] = df['tm2_80_game_avg_pace'] - df['nba_80_gm_avg_pace']
df['80gm_tm2_minus_nba_std_pace'] = df['tm2_80_game_std_pace'] - df['nba_80_gm_std_pace']

In [33]:

# 40 games
df['40gm_tm1_minus_nba_avg_offrtg'] = df['tm1_40_gm_avg_offrtg'] - df['nba_40_gm_avg_offrtg']
df['40gm_tm1_minus_nba_std_offrtg'] = df['tm1_40_gm_std_offrtg'] - df['nba_40_gm_std_offrtg']
df['40gm_tm1_minus_nba_avg_defrtg'] = df['tm1_40_gm_avg_defrtg'] - df['nba_40_gm_avg_defrtg']
df['40gm_tm1_minus_nba_std_defrtg'] = df['tm1_40_gm_std_defrtg'] - df['nba_40_gm_std_defrtg']
df['40gm_tm1_minus_nba_avg_pace'] = df['tm1_40_gm_avg_pace'] - df['nba_40_gm_avg_pace']
df['40gm_tm1_minus_nba_std_pace'] = df['tm1_40_gm_std_pace'] - df['nba_40_gm_std_pace']

#team 2
df['40gm_tm2_minus_nba_avg_offrtg'] = df['tm2_40_gm_avg_offrtg'] - df['nba_40_gm_avg_offrtg']
df['40gm_tm2_minus_nba_std_offrtg'] = df['tm2_40_gm_std_offrtg'] - df['nba_40_gm_std_offrtg']
df['40gm_tm2_minus_nba_avg_defrtg'] = df['tm2_40_gm_avg_defrtg'] - df['nba_40_gm_avg_defrtg']
df['40gm_tm2_minus_nba_std_defrtg'] = df['tm2_40_gm_std_defrtg'] - df['nba_40_gm_std_defrtg']
df['40gm_tm2_minus_nba_avg_pace'] = df['tm2_40_gm_avg_pace'] - df['nba_40_gm_avg_pace']
df['40gm_tm2_minus_nba_std_pace'] = df['tm2_40_gm_std_pace'] - df['nba_40_gm_std_pace']

# 20 games
df['20gm_tm1_minus_nba_avg_offrtg'] = df['tm1_20_gm_avg_offrtg'] - df['nba_20_gm_avg_offrtg']
df['20gm_tm1_minus_nba_std_offrtg'] = df['tm1_20_gm_std_offrtg'] - df['nba_20_gm_std_offrtg']
df['20gm_tm1_minus_nba_avg_defrtg'] = df['tm1_20_gm_avg_defrtg'] - df['nba_20_gm_avg_defrtg']
df['20gm_tm1_minus_nba_std_defrtg'] = df['tm1_20_gm_std_defrtg'] - df['nba_20_gm_std_defrtg']
df['20gm_tm1_minus_nba_avg_pace'] = df['tm1_20_gm_avg_pace'] - df['nba_20_gm_avg_pace']
df['20gm_tm1_minus_nba_std_pace'] = df['tm1_20_gm_std_pace'] - df['nba_20_gm_std_pace']

# team 2
df['20gm_tm2_minus_nba_avg_offrtg'] = df['tm2_20_gm_avg_offrtg'] - df['nba_20_gm_avg_offrtg']
df['20gm_tm2_minus_nba_std_offrtg'] = df['tm2_20_gm_std_offrtg'] - df['nba_20_gm_std_offrtg']
df['20gm_tm2_minus_nba_avg_defrtg'] = df['tm2_20_gm_avg_defrtg'] - df['nba_20_gm_avg_defrtg']
df['20gm_tm2_minus_nba_std_defrtg'] = df['tm2_20_gm_std_defrtg'] - df['nba_20_gm_std_defrtg']
df['20gm_tm2_minus_nba_avg_pace'] = df['tm2_20_gm_avg_pace'] - df['nba_20_gm_avg_pace']
df['20gm_tm2_minus_nba_std_pace'] = df['tm2_20_gm_std_pace'] - df['nba_20_gm_std_pace']

# 10 games
df['10gm_tm1_minus_nba_avg_offrtg'] = df['tm1_10_gm_avg_offrtg'] - df['nba_10_gm_avg_offrtg']
df['10gm_tm1_minus_nba_std_offrtg'] = df['tm1_10_gm_std_offrtg'] - df['nba_10_gm_std_offrtg']
df['10gm_tm1_minus_nba_avg_defrtg'] = df['tm1_10_gm_avg_defrtg'] - df['nba_10_gm_avg_defrtg']
df['10gm_tm1_minus_nba_std_defrtg'] = df['tm1_10_gm_std_defrtg'] - df['nba_10_gm_std_defrtg']
df['10gm_tm1_minus_nba_avg_pace'] = df['tm1_10_gm_avg_pace'] - df['nba_10_gm_avg_pace']
df['10gm_tm1_minus_nba_std_pace'] = df['tm1_10_gm_std_pace'] - df['nba_10_gm_std_pace']

# team 2
df['10gm_tm2_minus_nba_avg_offrtg'] = df['tm2_10_gm_avg_offrtg'] - df['nba_10_gm_avg_offrtg']
df['10gm_tm2_minus_nba_std_offrtg'] = df['tm2_10_gm_std_offrtg'] - df['nba_10_gm_std_offrtg']
df['10gm_tm2_minus_nba_avg_defrtg'] = df['tm2_10_gm_avg_defrtg'] - df['nba_10_gm_avg_defrtg']
df['10gm_tm2_minus_nba_std_defrtg'] = df['tm2_10_gm_std_defrtg'] - df['nba_10_gm_std_defrtg']
df['10gm_tm2_minus_nba_avg_pace'] = df['tm2_10_gm_avg_pace'] - df['nba_10_gm_avg_pace']
df['10gm_tm2_minus_nba_std_pace'] = df['tm2_10_gm_std_pace'] - df['nba_10_gm_std_pace']



In [34]:
df.to_csv('data/team/All_Things_7.csv')

### Add Predicted Scores via weighted averages

Hypothetically, a team will score $$ Pace * OffRtg $$ points in a game.

Thus, I want to make a "prediction" (non-machine learning) of what a team will score by using this algorithm:

$$ A_P = [(A_p * B_p) /2] * [A_o - (B_d - N_d)] $$

Where:
- A = Team A
- B = Team B
- N = Nba Average
- _P = points
- _p = pace
- _o = offrtg
- _d = defrtg

Further, this needs to be weighted. 
I think a heavier weighing for recency (20 games) may be the best way to go. 

I propose 25% for 80 day, 35% for 40 day, and 40% for 20 day.

In [35]:
df = pd.read_csv('data/team/All_Things_7.csv')

In [36]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,...,10gm_tm1_minus_nba_avg_defrtg,10gm_tm1_minus_nba_std_defrtg,10gm_tm1_minus_nba_avg_pace,10gm_tm1_minus_nba_std_pace,10gm_tm2_minus_nba_avg_offrtg,10gm_tm2_minus_nba_std_offrtg,10gm_tm2_minus_nba_avg_defrtg,10gm_tm2_minus_nba_std_defrtg,10gm_tm2_minus_nba_avg_pace,10gm_tm2_minus_nba_std_pace
0,0,0,0,CHI,CHI @ NYK,2022-12-23,W,48.0,118.0,45.0,...,1.542,2.274419,1.235133,-0.140519,1.984667,1.646129,-9.588,2.413288,-2.503867,-0.536774
1,1,1,15,DET,DET @ ATL,2022-12-23,L,48.0,105.0,37.0,...,3.642,-0.701588,0.270133,-1.502805,-0.345333,0.249448,3.752,-2.686070,1.016133,-0.403234
2,2,2,27,LAC,LAC @ PHI,2022-12-23,L,48.0,114.0,41.0,...,-5.308,1.451982,-0.948867,0.263939,-1.735333,-0.865254,-5.598,-0.573509,-1.124867,0.476165
3,3,3,26,ATL,ATL vs. DET,2022-12-23,W,48.0,130.0,49.0,...,3.752,-2.686070,1.016133,-0.403234,-1.575333,1.319387,3.642,-0.701588,0.270133,-1.502805
4,4,4,25,MEM,MEM @ PHX,2022-12-23,W,48.0,125.0,45.0,...,-10.158,-5.702117,2.377133,-1.435267,3.784667,2.961647,2.502,1.429577,-1.917867,-1.007809


In [37]:
# test one
df['Tm1_Points_Estimate_80gm'] = ((df['tm1_80_game_avg_pace'] + df['tm2_80_game_avg_pace'])/2) * ((df['tm1_80_game_avg_offrtg'] - (df['tm2_80_game_avg_defrtg'] - df['nba_80_gm_avg_defrtg'])))

In [38]:
# Team 1 Points Scored Estimate from 80 game averages
df['Tm1_Points_Estimate_80gm'] = ((df['tm1_80_game_avg_pace'] + df['tm2_80_game_avg_pace'])/2) * ((df['tm1_80_game_avg_offrtg'] - (df['tm1_80_game_avg_defrtg'] - df['nba_80_gm_avg_defrtg'])))
df['Tm2_Points_Estimate_80gm'] = ((df['tm1_80_game_avg_pace'] + df['tm2_80_game_avg_pace'])/2) * ((df['tm2_80_game_avg_offrtg'] - (df['tm2_80_game_avg_defrtg'] - df['nba_80_gm_avg_defrtg'])))
df['Estimate_Points_Difference_80gm'] = df['Tm1_Points_Estimate_80gm'] - df['Tm2_Points_Estimate_80gm']
# now for 40 game averages
df['Tm1_Points_Estimate_40gm'] = ((df['tm1_40_gm_avg_pace'] + df['tm2_40_gm_avg_pace'])/2) * ((df['tm1_40_gm_avg_offrtg'] - (df['tm1_40_gm_avg_defrtg'] - df['nba_40_gm_avg_defrtg'])))
df['Tm2_Points_Estimate_40gm'] = ((df['tm1_40_gm_avg_pace'] + df['tm2_40_gm_avg_pace'])/2) * ((df['tm2_40_gm_avg_offrtg'] - (df['tm2_40_gm_avg_defrtg'] - df['nba_40_gm_avg_defrtg'])))
df['Estimate_Points_Difference_40gm'] = df['Tm1_Points_Estimate_40gm'] - df['Tm2_Points_Estimate_40gm']
# now for 20 game averages
df['Tm1_Points_Estimate_20gm'] = ((df['tm1_20_gm_avg_pace'] + df['tm2_20_gm_avg_pace'])/2) * ((df['tm1_20_gm_avg_offrtg'] - (df['tm1_20_gm_avg_defrtg'] - df['nba_20_gm_avg_defrtg'])))
df['Tm2_Points_Estimate_20gm'] = ((df['tm1_20_gm_avg_pace'] + df['tm2_20_gm_avg_pace'])/2) * ((df['tm2_20_gm_avg_offrtg'] - (df['tm2_20_gm_avg_defrtg'] - df['nba_20_gm_avg_defrtg'])))
df['Estimate_Points_Difference_20gm'] = df['Tm1_Points_Estimate_20gm'] - df['Tm2_Points_Estimate_20gm']

In [39]:
# weighted average Estimate
df['Tm1_Points_Estimate_Weighted'] = df['Tm1_Points_Estimate_80gm'] * .25 + df['Tm1_Points_Estimate_40gm'] * .35 + df['Tm1_Points_Estimate_20gm'] * .4
df['Tm2_Points_Estimate_Weighted'] = df['Tm2_Points_Estimate_80gm'] * .25 + df['Tm2_Points_Estimate_40gm'] * .35 + df['Tm2_Points_Estimate_20gm'] * .4
df['Estimate_Points_Difference_Weighted'] = df['Tm1_Points_Estimate_Weighted'] - df['Tm2_Points_Estimate_Weighted']

In [40]:
df.to_csv('data/team/All_Things_8.csv')